# 1. Collect useful syslogs 

In [1]:
loginMsg = "<142>1 2020-06-03T23:19:26+04:00 pcs.fsdxb.net PulseSecure: - - - 2020-06-03 23:19:26 - ive - [10.1.1.221] ahmed(Users)[Users] - VPN Tunneling: Session started for user with IPv4 address 10.0.1.118, hostname DESKTOP-4G2B8S6"
logoutMsg = "<142>1 2020-06-03T23:20:01+04:00 pcs.fsdxb.net PulseSecure: - - - 2020-06-03 23:20:01 - ive - [10.1.1.221] ahmed(Users)[Users] - VPN Tunneling: Session ended for user with IPv4 address 10.0.1.118"

In [2]:
loginMsg

'<142>1 2020-06-03T23:19:26+04:00 pcs.fsdxb.net PulseSecure: - - - 2020-06-03 23:19:26 - ive - [10.1.1.221] ahmed(Users)[Users] - VPN Tunneling: Session started for user with IPv4 address 10.0.1.118, hostname DESKTOP-4G2B8S6'

In [3]:
logoutMsg

'<142>1 2020-06-03T23:20:01+04:00 pcs.fsdxb.net PulseSecure: - - - 2020-06-03 23:20:01 - ive - [10.1.1.221] ahmed(Users)[Users] - VPN Tunneling: Session ended for user with IPv4 address 10.0.1.118'

In [4]:
from syslogutils import getVars, test_re, syslogConfig

# 2. Identify fields to Extract

One of the Fields should include at least one key (internal ip or mac) to be used to update the respective endpoint properties, referred by that key.

Fields of interest in loginMsg: 

Main Selector: "VPN Tunneling: Session started for user with IPv4 address"

1. key - internal IP: 10.0.1.118 
2. username: ahmed 
3. Realm(s): Users
4. hostname: DESKTOP-4G2B8S6
5. remote-ip: 10.1.1.221
6. vpn-Gateway: pcs.fsdxb.net
7. date-time of Login: 2020-06-03 23:19:26 

Fields of interest in logoutMsg: 

Main Selector: "VPN Tunneling: Session ended for user with IPv4 address"

1. key - internal IP: 10.0.1.118 

# 3. Map syslogs to Regular Expressions  

In [5]:
### Breakdown of loginMsg => Adding Variables as Targets with << >> delimiters: 

loginTarget = "<142>1 2020-06-03T23:19:26+04:00 <<vpn_gateway>> PulseSecure: - - - <<vpn_login_date_time>> - ive - [<<vpn_remote_ip>>] <<vpn_username>>(Users)[<<vpn_realms>>] - VPN Tunneling: Session started for user with IPv4 address <<ip>>, hostname <<vpn_hostname>>"
    
### Breakdown of logoutMsg: 

logoutTarget = "<142>1 2020-06-03T23:20:01+04:00 pcs.fsdxb.net PulseSecure: - - - 2020-06-03 23:20:01 - ive - [10.1.1.221] ahmed(Users)[Users] - VPN Tunneling: Session ended for user with IPv4 address <<ip>>"


In [6]:
getVars(loginTarget)

['$fsapi_vpn_gateway',
 '$fsapi_vpn_login_date_time',
 '$fsapi_vpn_remote_ip',
 '$fsapi_vpn_username',
 '$fsapi_vpn_realms',
 '$ip',
 '$fsapi_vpn_hostname']

In [7]:
getVars(logoutTarget)

['$ip']

## 3.1. Use regex101.com to generate the Regular expression to match the required fields

## 3.1.a. Testing the Logout regex with both messages

In [8]:
logout_regex = r".* VPN Tunneling: Session ended for user with IPv4 address ([\d\.]+)"

In [9]:
test_re(logout_regex, loginMsg)

Not a matching Syslog!


In [10]:
test_re(logout_regex, logoutMsg)

Matching Syslog.
Group 1 found: 10.0.1.118


## 3.2. Interpreting the login Message

Base Selector: VPN Tunneling: Session started for user with IPv4 address 
Exercise & Innovate in selecting the fields - Recorded in live session

In [11]:
login_regex = r".* (.*) .*: - - - (.*) - .* - \[([\d\.]+)\] (.*)\(.*\)\[(.*)\] - VPN Tunneling: Session started for user with IPv4 address ([\d\.]+), hostname (.*)"


In [12]:
test_re(login_regex, logoutMsg)

Not a matching Syslog!


In [13]:
test_re(login_regex, loginMsg)

Matching Syslog.
Group 1 found: pcs.fsdxb.net
Group 2 found: 2020-06-03 23:19:26
Group 3 found: 10.1.1.221
Group 4 found: ahmed
Group 5 found: Users
Group 6 found: 10.0.1.118
Group 7 found: DESKTOP-4G2B8S6


In [14]:
getVars(loginTarget)

['$fsapi_vpn_gateway',
 '$fsapi_vpn_login_date_time',
 '$fsapi_vpn_remote_ip',
 '$fsapi_vpn_username',
 '$fsapi_vpn_realms',
 '$ip',
 '$fsapi_vpn_hostname']

In [15]:
len(getVars(loginTarget))

7

# 4. Auto-Generate the configuration Skeleton 

In [16]:
sconf = syslogConfig('PSecure')

In [17]:
print(sconf.config)

fstool syslog set_property config.type1.option.psecure "PSecure"
fstool syslog set_property config.type2.option.psecure "PSecure"
fstool syslog set_property config.type3.option.psecure "PSecure"
fstool service restart



In [18]:
sconf.add('psecure vpn Login', login_regex, loginTarget, loginMsg)
sconf.setOnline('psecure vpn Login', True)

Correctly added the Property for psecure vpn Login
$online status set to True for psecure vpn Login


In [19]:
sconf.add('psecure vpn Logout', logout_regex, logoutTarget, logoutMsg)
sconf.setOnline('psecure vpn Logout', False)

Correctly added the Property for psecure vpn Logout
$online status set to False for psecure vpn Logout


In [20]:
testMsg = "<142>1 Test Message from Python Client, ip:10.0.1.118, vpngateway:10.0.1.195"

testTarget = "<142>1 Test Message from Python Client, ip:<<ip>>, vpngateway:<<vpn_gateway>>"

testRegex = ".* Test Message from Python Client, ip:([\d\.]+), vpngateway:([\d\.]+)"

sconf.add('psecure vpn Test', testRegex, testTarget, testMsg)

Correctly added the Property for psecure vpn Test


In [21]:
print(sconf.finalConfig())

fstool syslog set_property config.type1.option.psecure "PSecure"
fstool syslog set_property config.type2.option.psecure "PSecure"
fstool syslog set_property config.type3.option.psecure "PSecure"
fstool service restart

fstool syslog set_property template.psecure_vpn_login.type "psecure"
fstool syslog set_property template.psecure_vpn_login.regexp ".* (.*) .*: - - - (.*) - .* - \[([\d\.]+)\] (.*)\(.*\)\[(.*)\] - VPN Tunneling: Session started for user with IPv4 address ([\d\.]+), hostname (.*)"
fstool syslog set_property template.psecure_vpn_login.properties  "\$fsapi_vpn_gateway,\$fsapi_vpn_login_date_time,\$fsapi_vpn_remote_ip,\$fsapi_vpn_username,\$fsapi_vpn_realms,\$ip,\$fsapi_vpn_hostname"
fstool syslog set_property template.psecure_vpn_login.set_true "\$online"

fstool syslog set_property template.psecure_vpn_logout.type "psecure"
fstool syslog set_property template.psecure_vpn_logout.regexp ".* VPN Tunneling: Session ended for user with IPv4 address ([\d\.]+)"
fstool syslog set_p

In [22]:
print(sconf.finalConfig(em=True))

fstool oneach fstool syslog set_property config.type1.option.psecure "PSecure"
fstool oneach fstool syslog set_property config.type2.option.psecure "PSecure"
fstool oneach fstool syslog set_property config.type3.option.psecure "PSecure"
fstool oneach fstool service restart

fstool oneach fstool syslog set_property template.psecure_vpn_login.type "psecure"
fstool oneach fstool syslog set_property template.psecure_vpn_login.regexp ".* (.*) .*: - - - (.*) - .* - \[([\d\.]+)\] (.*)\(.*\)\[(.*)\] - VPN Tunneling: Session started for user with IPv4 address ([\d\.]+), hostname (.*)"
fstool oneach fstool syslog set_property template.psecure_vpn_login.properties  "\$fsapi_vpn_gateway,\$fsapi_vpn_login_date_time,\$fsapi_vpn_remote_ip,\$fsapi_vpn_username,\$fsapi_vpn_realms,\$ip,\$fsapi_vpn_hostname"
fstool oneach fstool syslog set_property template.psecure_vpn_login.set_true "\$online"

fstool oneach fstool syslog set_property template.psecure_vpn_logout.type "psecure"
fstool oneach fstool syslo

# 5. Write the DEX Properties needed for Syslog Messages Parsing 
The output file can be then imported into the DEX Web Properties - using the precreated Account. 

In [23]:
# Write DEX Properties to a file using pre-defined DEX User to an output XML file
sconf.writeDexProperties('lab', 'vpn_dex_properties.xml')

Generating dexProperties for: vpn_gateway
Generating dexProperties for: vpn_login_date_time
Generating dexProperties for: vpn_remote_ip
Generating dexProperties for: vpn_username
Generating dexProperties for: vpn_realms
Generating dexProperties for: vpn_hostname
Generating dexProperties for: vpn_gateway
Writing DEX Properties to file: vpn_dex_properties.xml


True